In [3]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import random

# 1. Carregar dados
file_path = 'data/glass.data'
columns = ['Id','RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','Type_of_glass']
df = pd.read_csv(file_path, header=None, names=columns)
df.drop(columns=['Id'], inplace=True)

# 2. Backup original
df_original = df.copy()

# 3. Inserir valores ausentes na coluna categórica
random.seed(42)
missing_indices = df.sample(frac=0.2, random_state=42).index
df.loc[missing_indices, 'Type_of_glass'] = np.nan

# 4. Separar X e y
X = df.drop(columns=['Type_of_glass'])
y = df['Type_of_glass']

# 5. Transformar y para numérico (caso não seja)
y_encoded = y.copy()
if y.dtype == object or y.dtype.name == 'category':
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
else:
    y_encoded = y

# 6. Concatenar para imputação
df_imp = X.copy()
df_imp['Type_of_glass'] = y_encoded

# 7. Imputar usando IterativeImputer com RandomForestClassifier
# Como é categórico, usamos Round para garantir valores inteiros
imputer = IterativeImputer(estimator=RandomForestClassifier(), 
                           max_iter=10, random_state=42, 
                           sample_posterior=False)

df_imp_imputed = imputer.fit_transform(df_imp)

# 8. Recuperar o dataframe com imputação aplicada
df_imputed = pd.DataFrame(df_imp_imputed, columns=df_imp.columns)

# 9. Arredondar coluna alvo imputada e converter para int
df_imputed['Type_of_glass'] = df_imputed['Type_of_glass'].round().astype(int)

# 10. (Opcional) Reverter LabelEncoder se foi usado
if 'le' in locals():
    df_imputed['Type_of_glass'] = le.inverse_transform(df_imputed['Type_of_glass'])

# 11. Comparar original vs imputado (apenas nas posições que estavam ausentes)
acertos = (df_imputed.loc[missing_indices, 'Type_of_glass'].values == df_original.loc[missing_indices, 'Type_of_glass'].values)
taxa_acerto = np.mean(acertos)
print(f"Taxa de acerto da imputação: {taxa_acerto:.2%}")


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

### NÂO FAZ SENTIDO USAR MICE!

Por que não faz sentido usar MICE para uma única coluna?
O MICE é um método desenhado para múltiplas variáveis com valores ausentes, que se influenciam mutuamente. Quando apenas uma coluna possui valores faltantes:

- Um modelo supervisionado simples (como RandomForestClassifier, KNN, LogisticRegression, etc.) é mais eficiente e direto.

- O MICE se torna desnecessariamente complexo, pois ele simula um processo iterativo em que várias colunas seriam alternadamente "alvo" e "features", o que não ocorre quando há apenas uma com NaN.


---
O erro de "label type continuous" ao usar MICE com IterativeImputer e RandomForestClassifier acontece justamente porque:

O IterativeImputer espera que o estimador (o modelo que você passa) lide com os dados da coluna como se fossem contínuos (regressão), mas você está passando um classificador que precisa de rótulos categóricos e inteiros, sem valores ausentes no target.

Como só há uma coluna categórica com valores ausentes, o IterativeImputer tenta tratar tudo como contínuo e falha na hora de encaixar o classificador para essa coluna.

Além disso, o método é pensado para imputar várias colunas faltantes em sequência, não só uma.

Oso do MICE com IterativeImputer e RandomForestClassifier numa única coluna categórica com NaN é a causa dos erros!!